# Annex GG Full Demo — Simulation Workflow (Detectability Curves)

This notebook runs the complete Annex GG‑style workflow end‑to‑end using synthetic but realistic data that mimics an adult chest with a titanium spinal rod.  
It computes CHO decision values, **AUC vs. Dose**, **AUC vs. Contrast**, performs a **paired one‑tailed t‑test**, and reports **ΔAUC (MAR–FBP)** statistics.

> This demo prefers **detectability curves** (AUC vs. dose/contrast), not heatmaps.


In [ ]:
import os, yaml, numpy as np, matplotlib.pyplot as plt, pandas as pd
from pathlib import Path

from mareval import (
    generate_synthetic_study, extract_roi,
    build_pca_channels, apply_channels, cho_template, cho_decision_values,
    compute_auc_ci, paired_ttest_one_tailed,
)

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

## 1) Load Configuration

In [ ]:
# Try to load repo config; fall back to defaults
cfg_candidates = [
    Path('configs')/'adult_chest_spinal_rod.yaml',
    Path('../configs')/'adult_chest_spinal_rod.yaml',
]
cfg = None
for p in cfg_candidates:
    if p.exists():
        with open(p, 'r') as f:
            cfg = yaml.safe_load(f)
        print(f"Loaded config: {p}")
        break

if cfg is None:
    print("Config not found; using defaults.")
    cfg = {
        'image_shape': [128, 128],
        'lesion_radius_px': 3,
        'doses': 11,
        'contrasts': 7,
        'realizations': 10,
        'recon_types': ['FBP', 'MAR'],
        'classes': ['absent', 'present'],
        'base_bg_mean': 1000.0,
        'base_bg_std': 20.0,
        'dose_noise_scale': 1.0,
        'contrast_min': 1.0,
        'contrast_max': 1.06,
        'rng_seed': 7,
        'roi_size': [25, 25],
        'channels': 16,
        'lambda_reg': 1e-3,
    }

out_dir = Path('outputs')
out_dir_exists = out_dir.exists()
print(f"outputs/ exists: {out_dir_exists}")
cfg

## 2) Generate Synthetic Study

In [ ]:
study = generate_synthetic_study(cfg)
dose_levels = study['dose_levels']
contrast_levels = study['contrast_levels']
images = study['images']
roi_centers = study['roi_centers']

print('N dose:', len(dose_levels), 'N contrast:', len(contrast_levels))
print('Example key count:', len(images))

## 3) Visualize & Extract Training ROIs for Channels

In [ ]:
# Preview a couple of slices and the ROI box
roi_size = tuple(cfg['roi_size'])
im_keys = [
    (0, 0, 0, 'FBP', 'present'),
    (10, 6, 0, 'MAR', 'absent'),
]

fig, axes = plt.subplots(1, len(im_keys), figsize=(8, 4))
for ax, k in zip(axes, im_keys):
    ax.imshow(images[k], cmap='gray')
    cy, cx = roi_centers[0]
    h, w = roi_size
    rect = plt.Rectangle((cx - w//2, cy - h//2), w, h, fill=False, linewidth=1.5)
    ax.add_patch(rect)
    ax.set_title(str(k))
    ax.axis('off')
plt.show()

# Build a small training set (mix of conditions) and extract ROIs
train_imgs = [images[(0, 0, r, 'FBP', 'absent')] for r in range(3)] +              [images[(10, 6, r, 'MAR', 'absent')] for r in range(3)]

def extract_roi_single(image, center, size):
    return extract_roi(image, center, size)

train_rois = [extract_roi_single(im, roi_centers[0], roi_size).ravel() for im in train_imgs]
train_rois = np.vstack(train_rois)
train_rois.shape

## 4) Build PCA Channels & CHO Template

In [ ]:
U = build_pca_channels(train_rois, n_channels=int(cfg['channels']), whiten=True, random_state=0)

def build_cell_data(d, j, recon):
    pos = [extract_roi_single(images[(d, j, r, recon, 'present')], roi_centers[0], roi_size).ravel()
           for r in range(cfg['realizations'])]
    neg = [extract_roi_single(images[(d, j, r, recon, 'absent')],  roi_centers[0], roi_size).ravel()
           for r in range(cfg['realizations'])]
    return np.asarray(pos), np.asarray(neg)

# Example cell preview
pos_fbp, neg_fbp = build_cell_data(10, 6, 'FBP')
ch_pos = apply_channels(pos_fbp, U)
ch_neg = apply_channels(neg_fbp, U)
w = cho_template(ch_pos, ch_neg, lambda_reg=float(cfg.get('lambda_reg', 1e-3)))
w.shape

## 5) Detectability Curves (AUC vs. Dose, AUC vs. Contrast)

In [ ]:
# Compute AUC grid for both recon types using per-cell CHO templates
auc_fb  = np.zeros((len(dose_levels), len(contrast_levels)))
auc_mar = np.zeros_like(auc_fb)

for id_d, d in enumerate(dose_levels):
    for id_c, _ in enumerate(contrast_levels):
        # FBP
        pos, neg = build_cell_data(d, id_c, 'FBP')
        ch_pos = apply_channels(pos, U)
        ch_neg = apply_channels(neg, U)
        w = cho_template(ch_pos, ch_neg, lambda_reg=float(cfg.get('lambda_reg', 1e-3)))
        dv = np.concatenate([cho_decision_values(ch_pos, w), cho_decision_values(ch_neg, w)])
        labels = np.concatenate([np.ones(len(pos)), np.zeros(len(neg))])
        auc_fb[id_d, id_c] = float(compute_auc_ci(dv, labels)['auc'])

        # MAR
        pos, neg = build_cell_data(d, id_c, 'MAR')
        ch_pos = apply_channels(pos, U)
        ch_neg = apply_channels(neg, U)
        w = cho_template(ch_pos, ch_neg, lambda_reg=float(cfg.get('lambda_reg', 1e-3)))
        dv = np.concatenate([cho_decision_values(ch_pos, w), cho_decision_values(ch_neg, w)])
        labels = np.concatenate([np.ones(len(pos)), np.zeros(len(neg))])
        auc_mar[id_d, id_c] = float(compute_auc_ci(dv, labels)['auc'])

# AUC vs Dose (averaged over contrast)
mean_auc_fb_dose  = auc_fb.mean(axis=1)
mean_auc_mar_dose = auc_mar.mean(axis=1)

plt.figure(figsize=(6,4))
plt.plot(dose_levels, mean_auc_fb_dose, marker='o', label='FBP')
plt.plot(dose_levels, mean_auc_mar_dose, marker='o', label='MAR')
plt.xlabel('Dose (arb. units)')
plt.ylabel('AUC (CHO)')
plt.title('Detectability vs Dose (averaged over contrast)')
plt.ylim(0.5, 1.0)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# AUC vs Contrast (averaged over dose)
mean_auc_fb_con  = auc_fb.mean(axis=0)
mean_auc_mar_con = auc_mar.mean(axis=0)
contr_vals = np.array(contrast_levels)

plt.figure(figsize=(6,4))
plt.plot(contr_vals, mean_auc_fb_con, marker='o', label='FBP')
plt.plot(contr_vals, mean_auc_mar_con, marker='o', label='MAR')
plt.xlabel('Signal Contrast (× background)')
plt.ylabel('AUC (CHO)')
plt.title('Detectability vs Contrast (averaged over dose)')
plt.ylim(0.5, 1.0)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 6) Statistical Comparison & ΔAUC

In [ ]:
# Paired one-tailed t-test over the whole grid
delta_mean, p_val = paired_ttest_one_tailed(auc_mar.ravel(), auc_fb.ravel())
ΔAUC = auc_mar - auc_fb
bias_summary = {
    'delta_auc_mean': float(delta_mean),
    'p_value_one_tailed': float(p_val),
    'status': 'improved' if delta_mean > 0 else ('worse' if delta_mean < 0 else 'no_change')
}
bias_summary

## 7) Save CSV Summaries (optional)

In [ ]:
from pathlib import Path
out_dir = Path('outputs')
if out_dir.exists():
    import numpy as np, pandas as pd
    df_fb = pd.DataFrame(auc_fb, index=dose_levels, columns=np.round(contrast_levels, 3))
    df_mar = pd.DataFrame(auc_mar, index=dose_levels, columns=np.round(contrast_levels, 3))
    df_delta = pd.DataFrame(ΔAUC, index=dose_levels, columns=np.round(contrast_levels, 3))

    df_fb.to_csv(out_dir/'auc_fbp.csv')
    df_mar.to_csv(out_dir/'auc_mar.csv')
    df_delta.to_csv(out_dir/'delta_auc.csv')
    print("Saved: outputs/auc_fbp.csv, outputs/auc_mar.csv, outputs/delta_auc.csv")
else:
    print("outputs/ not present; skipping CSV export.")